# Homework 3

## Question 1

In [1]:
from bs4 import BeautifulSoup
import requests

def main() :
    letters = ['c','d','e','f','g','h','i','k','l','m','n','o','p','r','s','t','u','w','y','z']
    # A needs to be separately done because it is the home/first page.
    url = "http://www.palindromelist.net/"
    result = requests.get(url)
    doc = BeautifulSoup(result.text, "html.parser")
    palindromes = doc.find_all("p")[1]
    palindromes = palindromes.find_all("a")
    score = 0
    failed = ""
    count = 0
    for pali in palindromes:
        pali = pali.string
        if palindrome(pali) :
            score += 1
            count += 1
        else :
            count += 1
            failed += pali
    if failed :
        print(f'{url:50}{str(score) + "/" + str(count):10}{failed}')        
    else :
        print(f'{url:50}{str(score) + "/" + str(count):10}{"None"}')
    # B also needs to be separately done since, and ONLY for it, the first a element is empty.
    burl = "http://www.palindromelist.net/palindromes-b/"
    result = requests.get(burl)
    doc = BeautifulSoup(result.text, "html.parser")
    palindromes = doc.find_all("p")[1]
    palindromes = palindromes.find_all("a")
    palindromes = palindromes[1:]
    score = 0
    failed = ""
    count = 0
    for pali in palindromes:
        pali = pali.string
        if palindrome(pali) :
            score += 1
            count += 1
        else :
            count += 1
            failed += pali
    if failed :
        print(f'{burl:50}{str(score) + "/" + str(count):10}{failed}')        
    else :
        print(f'{burl:50}{str(score) + "/" + str(count):10}{"None"}')
    # ... NOW iterate through the rest of the letters.
    # If I were to put B and A into the loop below, then either empty elements
    # would be read or the first palindrome of EVERY page would get cut off.
    for letter in letters :
        newrl = f"{url}palindromes-{letter}/"
        result = requests.get(newrl)
        doc = BeautifulSoup(result.text, "html.parser")
        palindromes = doc.find_all("p")[1]
        palindromes = palindromes.find_all("a")
        score = 0
        failed = ""
        count = 0
        for pali in palindromes:
            pali = pali.string
            if palindrome(pali) :
                score += 1
                count += 1
            else :
                count += 1
                if failed :
                    failed += f'\n{"":60}{pali}'
                else :
                    failed += pali
        if failed :
            print(f'{newrl:50}{str(score) + "/" + str(count):10}{failed}')
        else :
            print(f'{newrl:50}{str(score) + "/" + str(count):10}{"None"}')

def palindrome(pali) :
    pali = pali.lower()
    ilap = ""
    for character in pali :
        if character.isalnum() :
            ilap += character
    reverse = ilap[::-1]
    return reverse == ilap

print("-"*96)
print(f'{"Source":50}{"Score":10}{"Failed Palindromes"}')
print("-"*96)
main()

------------------------------------------------------------------------------------------------
Source                                            Score     Failed Palindromes
------------------------------------------------------------------------------------------------
http://www.palindromelist.net/                    36/37     Aibohphobia (fear of palindromes)
http://www.palindromelist.net/palindromes-b/      9/9       None
http://www.palindromelist.net/palindromes-c/      7/8       Capac (Michigan)
http://www.palindromelist.net/palindromes-d/      58/58     None
http://www.palindromelist.net/palindromes-e/      31/31     None
http://www.palindromelist.net/palindromes-f/      1/1       None
http://www.palindromelist.net/palindromes-g/      17/18     Glenelg (cities in Ontario, Maryland, and Austrailia)
http://www.palindromelist.net/palindromes-h/      11/11     None
http://www.palindromelist.net/palindromes-i/      20/20     None
http://www.palindromelist.net/palindromes-k/      2/

## Question 2

In [36]:
from lxml import html
import requests
from datetime import datetime

def print_formatted_table(data_table):
    print(f'{"Date":30}{"Event"}')
    print("-"*110)
    for dayte in data_table :
        if "Sept" in dayte[0] :
            dayte[0] = dayte[0].replace("Sept", "Sep")
        if '-' in dayte[0] :
            dayte[0] = dayte[0].split('-')
            d = datetime.strptime(dayte[0][0], "%B %d")
            d2 = datetime.strptime(dayte[0][1], "%d")
            print(f'{d.strftime("%m/%d - "):8}{d.strftime("%m/")}{d2.strftime("%d"):19}{dayte[1]:.80}')
            continue
        if '.' in dayte[0] :
            d = datetime.strptime(dayte[0], "%b. %d")
            print(f'{d.strftime("%m/%d"):30}{dayte[1]:.80}')
            continue
        else :
            d = datetime.strptime(dayte[0], "%B %d")
            print(f'{d.strftime("%m/%d"):30}{dayte[1]:.80}')
            continue

def get_html(url):
    response = requests.get(url)  # get page data from server, block redirects
    source_code = response.content # get string of source code from response
    return source_code

def scrape_data(url):
    return get_data_table(get_html(url))

def get_data_table(source_code):
    data_table = []
    html_elem = html.document_fromstring(source_code)  # make HTML element object
    tables = html_elem.cssselect("table")
    if len(tables) > 3:
        table0 = tables[0]
        # a tr is a HTML table row; selecting tr selects all rows
        trs = table0.cssselect('tr')
        # go through each row in the table
        for tr in trs:
            output_table_row = []
            tds = tr.cssselect('td')  # a td is a HTML table cell; selecting td selects a group of cells
            # go through each cell and grab the text content
            for td in tds:
                cell_text = td.text_content().strip() # always practice to strip away extra whitespace
                output_table_row.append(cell_text)
            data_table.append(output_table_row)
    return data_table  

def main():
    url = "https://www.fit.edu/registrar/academic-calendar/spring-2022/"
    data_table = scrape_data(url)
    data_table = data_table[1:]
    print_formatted_table(data_table)

main()

Date                          Event
--------------------------------------------------------------------------------------------------------------
09/10                         Last day to Petition to Graduate for Spring Semester 2022 without a late fee
11/07                         Registration for Melbourne Campus Spring Semester 2022 begins at 8pm
11/10                         Registration for Education Centers Spring Semester 2022 begins
01/03                         Tuition and fees due for Spring Semester 2022
01/10                         CLASSES BEGIN (Monday)
01/10                         Last day for returning students to register for Spring 2022 without a late regis
01/17                         Holiday (Martin Luther King Jr. Day) (NO CLASSES)
01/19                         Last day to register or add a class, or drop a class with full tuition refund an
01/28                         Re-petition deadline for Spring Semester 2022 (for students who had petitioned f
01/31       